In [16]:
#!/usr/bin/env python
# -*- coding: latin-1 -*-
import pandas as pd # pandas is a data manipulation library
import numpy as np #provides numerical arrays and functions to manipulate the arrays efficiently
import re
import math
import csv
from SPARQLWrapper import SPARQLWrapper,JSON, SPARQLWrapper2
from collections import defaultdict
from tqdm import tqdm 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

import functools

<ipython-input-16-3d86a0ffe6a7>:18: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
# load movies.csv
titles_df = pd.read_csv('dataset/movies.tsv', sep='\t',  lineterminator='\n', names=None)

In [17]:
titles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7292281 entries, 0 to 7292280
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         int64 
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: int64(1), object(8)
memory usage: 500.7+ MB


In [ ]:
titles_df.info()

In [ ]:
titles_df.titleType.unique()

In [ ]:
titles_df.groupby('titleType')['primaryTitle'].count()

In [18]:
#Keep only titles of type movie
movies_df = titles_df[titles_df['titleType'] == 'movie'] 

In [19]:
len(movies_df)

557139

In [21]:
imdb_movies_df = movies_df[['tconst','primaryTitle','isAdult','startYear','genres']].copy()

In [22]:
imdb_movies_df = imdb_movies_df[imdb_movies_df['genres'] != '\\N']

In [23]:
len(imdb_movies_df)

487194

In [24]:
#keep only 2 columns id and title
movies_titles_df = imdb_movies_df[['tconst','primaryTitle','genres']]

In [25]:
#rename the columns
movies_titles_df = movies_titles_df.rename(columns= {'tconst':'movie_id', 'primaryTitle':'movie_name'})
movies_titles_df.describe

<bound method NDFrame.describe of           movie_id                                                movie_name                 genres
8        tt0000009  Miss Jerry                                                Romance              
331      tt0000335  Soldiers of the Cross                                     Biography,Drama      
570      tt0000574  The Story of the Kelly Gang                               Biography,Crime,Drama
610      tt0000615  Robbery Under Arms                                        Drama                
625      tt0000630  Hamlet                                                    Drama                
...            ...     ...                                                      ...                
7292169  tt9916620  The Copeland Case                                         Drama                
7292170  tt9916622  Rodolpho Teóphilo - O Legado de um Pioneiro               Documentary          
7292197  tt9916680  De la ilusión al desconcierto: cine colombiano

In [26]:
# load names file which contain the names for all crew members
crew_names_df = pd.read_csv('DataSet/actors.tsv', sep='\t',  lineterminator='\n', names=None)

In [27]:
#keep only the name and id of the crews
crew_names_df= crew_names_df[['nconst','primaryName']].copy()
crew_names_df = crew_names_df.rename(columns={'nconst':'crew_id', 'primaryName':'crew_name'})
crew_names_df.describe

<bound method NDFrame.describe of             crew_id           crew_name
0         nm0000001  Fred Astaire      
1         nm0000002  Lauren Bacall     
2         nm0000003  Brigitte Bardot   
3         nm0000004  John Belushi      
4         nm0000005  Ingmar Bergman    
...             ...             ...    
10464396  nm9993714  Romeo del Rosario 
10464397  nm9993716  Essias Loberg     
10464398  nm9993717  Harikrishnan Rajan
10464399  nm9993718  Aayush Nair       
10464400  nm9993719  Andre Hill        

[10464401 rows x 2 columns]>

In [ ]:
#load the cast data which contsins mapping between the movie id and the cast (actor, director,writer) ids
cast_df = pd.read_csv('C:/Users/rania/OneDrive/AI Master/Project/Python_Code/DataSet/actors_movies_ids.tsv', sep='\t',  lineterminator='\n', names=None)

In [28]:
cast_df.describe


<bound method NDFrame.describe of              tconst  ordering     nconst         category                      job                 characters
0         tt0000001  1         nm1588970  self             \N                       ["Self"]                 
1         tt0000001  2         nm0005690  director         \N                       \N                       
2         tt0000001  3         nm0374658  cinematographer  director of photography  \N                       
3         tt0000002  1         nm0721526  director         \N                       \N                       
4         tt0000002  2         nm1335271  composer         \N                       \N                       
...             ... ..               ...       ...         ..                       ..                       
41706001  tt9916880  5         nm0996406  director         principal director       \N                       
41706002  tt9916880  6         nm1482639  writer           \N                       \N

In [29]:
#check the available cast categories  
cast_df['category'].unique()

array(['self', 'director', 'cinematographer', 'composer', 'producer',
       'editor', 'actor', 'actress', 'writer', 'production_designer',
       'archive_footage', 'archive_sound'], dtype=object)

In [30]:
#copy only the actors and actresess data
starring_df = cast_df[cast_df['category'].isin(['actor','actress'])].copy()

In [31]:
starring_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16735395 entries, 11 to 41706005
Data columns (total 6 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   tconst      object
 1   ordering    int64 
 2   nconst      object
 3   category    object
 4   job         object
 5   characters  object
dtypes: int64(1), object(5)
memory usage: 893.8+ MB


In [32]:
#keep only the movie id and the actor id
starring_df = starring_df[['tconst','nconst']].copy()

In [33]:
#rename columns to actor_id and movie_id
starring_df = starring_df.rename(columns={'tconst':'movie_id', 'nconst':'actor_id'})

In [34]:
#get the actors name for each movie from the crew names list
movie_id_actor_name_df = starring_df.merge(crew_names_df, how="inner", left_on='actor_id', right_on='crew_id')

In [35]:
movie_id_actor_name_df= movie_id_actor_name_df.rename(columns={'crew_name':'actor_name'})
movie_id_actor_name_df.drop(columns={'crew_id'}, inplace= True)

In [36]:
movie_id_actor_name_df.describe

<bound method NDFrame.describe of            movie_id    actor_id        actor_name
0         tt0000005  nm0443482   Charles Kayser  
1         tt0000005  nm0653042   John Ott        
2         tt0000007  nm0179163   James J. Corbett
3         tt0003116  nm0179163   James J. Corbett
4         tt0003730  nm0179163   James J. Corbett
...             ...        ...                ...
16709631  tt9916756  nm10781824  Lakeisha Barnes 
16709632  tt9916764  nm10538641  Yvonne Akins    
16709633  tt9916856  nm10538650  Anna Fitoussi   
16709634  tt9916856  nm10538646  Andreas Demmel  
16709635  tt9916856  nm10538647  Kathrin Knöpfle 

[16709636 rows x 3 columns]>

In [37]:
actors_grouped = movie_id_actor_name_df.groupby(['movie_id'], as_index=False)
#aggregate the directors and directors in one column for each separated by |
movie_id_actor_name_grouped_df = actors_grouped.agg({
                                        'actor_name':lambda x: '|'.join(set(x))
                                        })
#save grouped data to file
movie_id_actor_name_grouped_df.to_csv("generated_dataset/actors_movies_id_mapping.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'generated_dataset/actors_movies_id_mapping.csv'

In [38]:
final_dataset_df = movies_titles_df.merge(movie_id_actor_name_grouped_df, how="inner", on='movie_id')

In [39]:
#copy only the directors data
director_df = cast_df[cast_df['category'].isin(['director'])].copy()
director_df.describe

<bound method NDFrame.describe of              tconst  ordering      nconst  category                 job characters
1         tt0000001  2         nm0005690   director  \N                  \N       
3         tt0000002  1         nm0721526   director  \N                  \N       
5         tt0000003  1         nm0721526   director  \N                  \N       
9         tt0000004  1         nm0721526   director  \N                  \N       
13        tt0000005  3         nm0005690   director  \N                  \N       
...             ... ..               ...        ...  ..                  ..       
41705974  tt9916850  6         nm5519375   director  \N                  \N       
41705983  tt9916852  5         nm5519454   director  \N                  \N       
41705984  tt9916852  6         nm5519375   director  \N                  \N       
41705992  tt9916856  5         nm10538645  director  \N                  \N       
41706001  tt9916880  5         nm0996406   director  

In [40]:
#keep only the movie id and the actor id
director_df = director_df[['tconst','nconst']].copy()
#rename columns to actor_id and movie_id
director_df = director_df.rename(columns={'tconst':'movie_id', 'nconst':'director_id'})
director_df.describe

<bound method NDFrame.describe of            movie_id director_id
1         tt0000001  nm0005690 
3         tt0000002  nm0721526 
5         tt0000003  nm0721526 
9         tt0000004  nm0721526 
13        tt0000005  nm0005690 
...             ...        ... 
41705974  tt9916850  nm5519375 
41705983  tt9916852  nm5519454 
41705984  tt9916852  nm5519375 
41705992  tt9916856  nm10538645
41706001  tt9916880  nm0996406 

[4807068 rows x 2 columns]>

In [41]:
#get the actors name for each movie from the crew names list
movie_director_name_df = director_df.merge(crew_names_df, how="inner", left_on='director_id', right_on='crew_id')


In [42]:
movie_director_name_df= movie_director_name_df.rename(columns={'crew_name':'director_name'})
#movie_id_actor_name_df.drop(columns={'crew_id'}, inplace= True)

In [43]:
directors_grouped = movie_director_name_df.groupby(['movie_id'], as_index=False)
#aggregate the directors and directors in one column for each separated by |
movie_id_director_name_grouped_df = directors_grouped.agg({
                                        'director_name':lambda x: '|'.join(set(x))
                                        
                                        })
#save grouped data to file
movie_id_director_name_grouped_df.to_csv("generated_dataset/directors_movies_id_mapping.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'generated_dataset/directors_movies_id_mapping.csv'

In [44]:
len(movie_id_director_name_grouped_df.movie_id.unique())

3769487

In [45]:
final_dataset_df = final_dataset_df.merge(movie_id_director_name_grouped_df, how="inner", on='movie_id')

In [46]:
final_dataset_df

,movie_id,movie_name,genres,actor_name,director_name
0,tt0000009,Miss Jerry,Romance,William Courtenay|Blanche Bayliss|Chauncey Depew,Alexander Black
1,tt0000335,Soldiers of the Cross,"Biography,Drama",John Jones|Orrie Perry|Reg Perry|Beatrice Day|Mr. Graham|Harold Graham,Herbert Booth|Joseph Perry
2,tt0000574,The Story of the Kelly Gang,"Biography,Crime,Drama",Norman Campbell|Bella Cola|John Tait|Elizabeth Tait,Charles Tait
3,tt0000615,Robbery Under Arms,Drama,Jim Gerald|George Merriman|Lance Vane|Arthur Guest|Rhoda Dendron|William Duff,Charles MacMahon
4,tt0000630,Hamlet,Drama,Fernanda Negri Pouget,Mario Caserini
...,...,...,...,...,...
322228,tt9916362,Coven,"Drama,History,Horror",Alex Brendemühl|Daniel Fanego|Amaia Aberasturi|Garazi Urkola,Pablo Agüero
322229,tt9916428,The Secret of China,"Adventure,History,War",Valery Gadreau|A.C Stokes|Vincent Matile|Kenan Heppe|Wang Peng Kai|Shenyang Xiao|Wenli Jiang,Jixing Wang
322230,tt9916538,Kuambil Lagi Hatiku,Drama,Cut Mini Theo|Sahil Shah|Lala Karmela|Ria Irawan,Azhar Kinoi Lubis
322231,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Documentary,Oldair Soares Ammom,Angela Gurgel|Ana Célia de Oliveira


In [47]:
#copy only the directors data
writer_df = cast_df[cast_df['category'].isin(['writer'])].copy()
writer_df.describe

<bound method NDFrame.describe of              tconst  ordering     nconst category                     job characters
90        tt0000036  3         nm0410331  writer   story "Rip Van Winkle"  \N       
147       tt0000076  3         nm0410331  writer   story "Rip Van Winkle"  \N       
189       tt0000108  3         nm0410331  writer   story "Rip Van Winkle"  \N       
193       tt0000109  3         nm0410331  writer   story "Rip Van Winkle"  \N       
197       tt0000110  3         nm0410331  writer   story "Rip Van Winkle"  \N       
...             ... ..               ...     ...                      ...  ..       
41705986  tt9916852  8         nm1628284  writer   creator                 \N       
41705987  tt9916852  9         nm2921377  writer   \N                      \N       
41705993  tt9916856  6         nm6951431  writer   screenplay by           \N       
41706002  tt9916880  6         nm1482639  writer   \N                      \N       
41706003  tt9916880  7         

In [48]:
#keep only the movie id and the actor id
writer_df = writer_df[['tconst','nconst']].copy()
#rename columns to actor_id and movie_id
writer_df = writer_df.rename(columns={'tconst':'movie_id', 'nconst':'writer_id'})
writer_df.describe

<bound method NDFrame.describe of            movie_id  writer_id
90        tt0000036  nm0410331
147       tt0000076  nm0410331
189       tt0000108  nm0410331
193       tt0000109  nm0410331
197       tt0000110  nm0410331
...             ...        ...
41705986  tt9916852  nm1628284
41705987  tt9916852  nm2921377
41705993  tt9916856  nm6951431
41706002  tt9916880  nm1482639
41706003  tt9916880  nm2586970

[5519070 rows x 2 columns]>

In [49]:
#get the actors name for each movie from the crew names list
movie_writer_name_df = writer_df.merge(crew_names_df, how="inner", left_on='writer_id', right_on='crew_id')


In [50]:
movie_writer_name_df= movie_writer_name_df.rename(columns={'crew_name':'writer_name'})
#movie_id_actor_name_df.drop(columns={'crew_id'}, inplace= True)

In [51]:
writers_grouped = movie_writer_name_df.groupby(['movie_id'], as_index=False)
#aggregate the directors and directors in one column for each separated by |
movie_id_writer_name_grouped_df = writers_grouped.agg({
                                        'writer_name':lambda x: '|'.join(set(x))
                                        
                                        })
#save grouped data to file
movie_id_writer_name_grouped_df.to_csv("generated_dataset/writer_movies_id_mapping.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'generated_dataset/writer_movies_id_mapping.csv'

In [52]:
len(movie_id_writer_name_grouped_df.movie_id.unique())

2644140

In [53]:
final_dataset_df = final_dataset_df.merge(movie_id_writer_name_grouped_df, how="inner", on='movie_id')

In [57]:
len(final_dataset_df.movie_id.unique())

187650

In [58]:
final_dataset_df.to_csv('generated_dataset/imdb_final_dataset.csv', index= False)